In [1]:
from sklearn.externals import joblib
from nltk.tokenize import word_tokenize
from pandas import DataFrame, ExcelFile, read_csv, read_excel

In [2]:
def get_taleo_database(excel_path, sheet):
    xls = ExcelFile(excel_path, sep = ';', encoding = 'utf-8')
    df = read_excel(xls, sheet)
    return df

In [3]:
def tokenize_text(text):
    return word_tokenize(text)

In [4]:
def transform_data_to_tfidf(texts, tfidf_model):
    strs = []
    for text in texts:
        stra = tokenize_text(text)
        stra = ' '.join(stra)
        strs.append(stra)
    response = tfidf_model.transform(strs)
    return response

In [44]:
def categorize_items(line, section, section_identifier, tfidf_model):
    items = line.split(' - ')[1:]
    if items:
        items_tfidf = transform_data_to_tfidf(items, tfidf_model)
        labels = [section_identifier.predict(items_tfidf) for item in items]
        return ('$$$%%%&&&'.join([item for item, label in zip(items, labels[0]) if label == section]))
    return None

In [23]:
tfidf_model = joblib.load('../others/tfidf_model2019-03-05 183214.924115.pkl')
section_identifier = joblib.load('../others/classifier2019-03-05 183214.924115.pkl')

In [48]:
taleo_db = get_taleo_database('../2016 - 2018 Taleo Job Postings - Updated.xlsx', 'All')

In [52]:
output_df = DataFrame()

In [57]:
output_df['Position Title:'] = taleo_db['Requisition Title']
output_df['Department:'] = taleo_db['Job Family']
taleo_db['External: Responsibilities'] = taleo_db['External: Responsibilities'].fillna('')
output_df['responsibilities'] = taleo_db['External: Responsibilities'].apply(lambda x: ''.join(x.strip().split('-')[1:]) if x else None)
output_df['POSITION SUMMARY:'] = taleo_db['Original Description Section - External']

In [58]:
output_df['responsibilities'][0]

' Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit.  Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program.  Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors.  Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills.  Oversee the Stormwater Management Plan and the Biosolids Management Plan.  Manage the permitting process for the Deepwell Injection Project.  Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery.  Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental sampling and testing activities.  Assist in adapting procedures, processe

In [59]:
a=output_df['responsibilities'][0].split('.')

In [60]:
a

[' Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit',
 ' ',
 '  Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program',
 ' ',
 '  Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors',
 ' ',
 '  Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills',
 ' ',
 '  Oversee the Stormwater Management Plan and the Biosolids Management Plan',
 ' ',
 '  Manage the permitting process for the Deepwell Injection Project',
 ' ',
 '  Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery',
 ' ',
 '  Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental sa

In [15]:
resp_list1 = [i.strip() for i in a]

In [16]:
resp_list1

['Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit.',
 'Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program.',
 'Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors.',
 'Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills.',
 'Oversee the Stormwater Management Plan and the Biosolids Management Plan.',
 'Manage the permitting process for the Deepwell Injection Project.',
 'Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery.',
 'Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental sampling and testing activities.',
 'Assist in adapt

In [17]:
b=[]
for i in a:
    b.append(i.strip())

In [18]:
b

['Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit.',
 'Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program.',
 'Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors.',
 'Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills.',
 'Oversee the Stormwater Management Plan and the Biosolids Management Plan.',
 'Manage the permitting process for the Deepwell Injection Project.',
 'Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery.',
 'Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental sampling and testing activities.',
 'Assist in adapt

In [62]:
output_df['Position Title:'] = taleo_db['Requisition Title']
output_df['Department:'] = taleo_db['Job Family']
taleo_db['External: Responsibilities'] = taleo_db['External: Responsibilities'].fillna('',inplace=True)
output_df['responsibilities'] = taleo_db['External: Responsibilities'].apply(lambda x: ''.join(x.strip().split('-')[1:]) if x else None)
output_df['POSITION SUMMARY:'] = taleo_db['Original Description Section - External'] 
output_df['Date Revised:  (Select today’s date)'] = taleo_db['Req. Creation Date']
taleo_db['Qualifications - External'] = taleo_db['Qualifications - External'].fillna('')
output_df['CA'] =  taleo_db['Qualifications - External'].apply(lambda x: categorize_items(x, 'CA', section_identifier, tfidf_model))
'''output_df['CA_ASSET'] = ''
output_df['CA_REQD'] = '''''
output_df['CP'] =  taleo_db['Qualifications - External'].apply(lambda x: categorize_items(x, 'CP', section_identifier, tfidf_model))
'''output_df['CP_ASSET'] = ''
output_df['CP_REQD'] = '''''
output_df['EI'] =  taleo_db['Qualifications - External'].apply(lambda x: categorize_items(x, 'EI', section_identifier, tfidf_model))
'''output_df['EI_ASSET'] = ''
output_df['EI_REQD'] = '''''
output_df['ER'] =  taleo_db['Qualifications - External'].apply(lambda x: categorize_items(x, 'ER', section_identifier, tfidf_model))
'''output_df['ER_ASSET'] = ''
output_df['ER_REQD'] = '''''
output_df['TS'] =  taleo_db['Qualifications - External'].apply(lambda x: categorize_items(x, 'TS', section_identifier, tfidf_model))
'''output_df['TS_ASSET'] = ''
output_df['TS_REQD'] = ''
output_df['Location (city/site):'] = ''
output_df['Reports To (Position):'] = ''
output_df['Safety-Sensitive:'] = ''
output_df['Title (and #) of Direct Reports:'] = ''
output_df['competency_group'] = ''
output_df['Bending/Crouching:'] = ''
output_df['Climbing:'] = ''
output_df['Driving:'] = ''
output_df['Keyboarding:'] = ''
output_df['Kneeling/Crawling:'] = ''
output_df['On-call:'] = ''
output_df['Operating equipment:'] = ''
output_df['Sedentary/Sitting:'] = ''
output_df['Shift work:'] = ''
output_df['Travel:'] = ''
output_df['Walking:'] = ''
output_df['Manual tools:'] = ''
output_df['Office equipment:'] = ''
output_df['Pneumatic tools:'] = ''
output_df['Power tools:'] = ''
output_df['Shop tools:'] = ''
output_df['Tool belt worn:'] = ''
output_df['Vibration tools:'] = ''
output_df['Welding:'] = ''
output_df[' Extreme heat/cold:'] = ''
output_df['Chemicals:'] = ''
output_df['Confined spaces:'] = ''
output_df['Heights:'] = ''
output_df['Moving equipment:'] = ''
output_df['Night time:'] = ''
output_df['Noise:'] = ''
output_df['Outdoors:'] = ''
output_df['Rotating equipment:'] = ''
output_df['Toxic gases:'] = ''
output_df['Uneven surfaces:'] = ''
output_df['Wet or damp:'] = ''
output_df['Light 11-20 pounds:'] = ''
output_df['Medium 21-50 pounds:'] = ''
output_df['Sedentary 0-10 pounds:'] = ''
output_df['Carrying:'] = ''
output_df['Lifting:'] = ''
output_df['Pushing/pulling:'] = ''
output_df['Standing:'] = '''''

"output_df['TS_ASSET'] = ''\noutput_df['TS_REQD'] = ''\noutput_df['Location (city/site):'] = ''\noutput_df['Reports To (Position):'] = ''\noutput_df['Safety-Sensitive:'] = ''\noutput_df['Title (and #) of Direct Reports:'] = ''\noutput_df['competency_group'] = ''\noutput_df['Bending/Crouching:'] = ''\noutput_df['Climbing:'] = ''\noutput_df['Driving:'] = ''\noutput_df['Keyboarding:'] = ''\noutput_df['Kneeling/Crawling:'] = ''\noutput_df['On-call:'] = ''\noutput_df['Operating equipment:'] = ''\noutput_df['Sedentary/Sitting:'] = ''\noutput_df['Shift work:'] = ''\noutput_df['Travel:'] = ''\noutput_df['Walking:'] = ''\noutput_df['Manual tools:'] = ''\noutput_df['Office equipment:'] = ''\noutput_df['Pneumatic tools:'] = ''\noutput_df['Power tools:'] = ''\noutput_df['Shop tools:'] = ''\noutput_df['Tool belt worn:'] = ''\noutput_df['Vibration tools:'] = ''\noutput_df['Welding:'] = ''\noutput_df[' Extreme heat/cold:'] = ''\noutput_df['Chemicals:'] = ''\noutput_df['Confined spaces:'] = ''\noutput

In [40]:
a=[""]

In [41]:
len(a)

1

In [42]:
a==[""]

True

In [43]:
b=a[0]

In [44]:
b

''

In [63]:
output_df.to_csv("profiles1.csv",sep=',',index=False)